In [293]:
import pandas as pd
import numpy as np
import datetime
import os

# Read in Data

In [296]:
cwd = '../data/raw_data/'

t = pd.read_csv('../data/march_to_september.csv') #read in data
mobility = pd.read_csv(cwd+'2020_US_Region_Mobility_Report.csv') #read in data
mobility.head()
max(mobility['date'])

'2020-12-01'

# Mobility Growth Rate

In [297]:
mobility['date'] = pd.to_datetime(mobility['date']) #convert str object to datetime
date_after = pd.to_datetime('2020-03-20') #this is where our twitter data begins
date_before = pd.to_datetime('2020-12-01') #this is where our twitter data ends
mobility = mobility[mobility['date'] >= date_after]
mobility = mobility[mobility['date'] <= date_before]
mobility = mobility.rename(columns={"retail_and_recreation_percent_change_from_baseline": "retail_rec", 
                                    "grocery_and_pharmacy_percent_change_from_baseline": "groc_pharm",
                                   "parks_percent_change_from_baseline": "parks",
                                   "transit_stations_percent_change_from_baseline": "transit",
                                   "workplaces_percent_change_from_baseline": "workplaces",
                                   "residential_percent_change_from_baseline": "residential"})
ca_mobility = mobility[mobility.sub_region_1 == 'California'] #get just mobility for california
ca_mobility.head(5)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_rec,groc_pharm,parks,transit,workplaces,residential
43409,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-20,-40.0,-3.0,-22.0,-46.0,-44.0,21.0
43410,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-21,-48.0,-13.0,-22.0,-47.0,-33.0,16.0
43411,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-22,-50.0,-22.0,-24.0,-52.0,-36.0,14.0
43412,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-23,-41.0,-18.0,-28.0,-52.0,-49.0,21.0
43413,US,United States,California,NaN,NaN,US-CA,NaN,2020-03-24,-42.0,-18.0,-31.0,-52.0,-50.0,22.0


In [298]:
ca = ca_mobility.drop(columns = ['census_fips_code','iso_3166_2_code', 'metro_area', 'country_region_code', 'country_region', 'sub_region_1'])
ca = ca.rename(columns = {"sub_region_2": "county"}) #rename for cols to match
ca = ca[ca['county'].notna()] #get rid of state level data

ca['public'] = (ca.workplaces + ca.transit + ca.groc_pharm + ca.retail_rec)/4

# Clean up Tweets

In [299]:
t.head(5)

,created_at,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,lang,longitude,latitude,state,county
0,2020-03-20,Been having so much fun with these live stream...,"[33.98902523, -118.35189342]","{'id': '3b77caf94bfc81fe', 'url': 'https://api...",False,"[0, 237]","{'id': 202073462, 'id_str': '202073462', 'name...",1241080025371144192,"{'hashtags': [{'text': 'live', 'indices': [168...",False,en,-118.351893,33.989025,California,Los Angeles County
1,2020-03-20,Should I get myself some Corona? @ CVS Pharmac...,"[37.67867687, -120.95931306]","{'id': 'e41805d7248dbf1e', 'url': 'https://api...",False,"[0, 71]","{'id': 19253082, 'id_str': '19253082', 'name':...",1241075581598433281,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-120.959313,37.678677,California,Stanislaus County
2,2020-03-20,Forced vacation? Social distancing? Unemployed...,"[33.8362, -117.89]","{'id': '0c2e6999105f8070', 'url': 'https://api...",False,"[0, 236]","{'id': 767209483, 'id_str': '767209483', 'name...",1241068075425857541,"{'hashtags': [{'text': 'chillin', 'indices': [...",True,en,-117.890000,33.836200,California,Orange County
3,2020-03-20,"If you have to leave your house for supplies, ...","[38.5763, -121.4927]","{'id': 'b71fac2ee9792cbe', 'url': 'https://api...",False,"[0, 231]","{'id': 1359353262, 'id_str': '1359353262', 'na...",1241065882694156288,"{'hashtags': [{'text': 'bronzed', 'indices': [...",False,en,-121.492700,38.576300,California,Sacramento County
4,2020-03-20,We just got baby chicks🐥🐤\nWe'll be having mor...,"[38.44279, -121.94901]","{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api...",False,"[0, 220]","{'id': 4113352096, 'id_str': '4113352096', 'na...",1241058413754417153,"{'hashtags': [{'text': 'lockewoodacres', 'indi...",False,en,-121.949010,38.442790,California,Solano County


In [300]:
missing_county = t[t['county'].isna()]

In [301]:
#riverside bounds 
lat = (33.447181, 34.078074)
long = (-117.674869, -114.517813)

long_boolean = missing_county['longitude'].between(long[0], long[1],  inclusive=True)
lat_boolean = missing_county['latitude'].between(lat[0], lat[1],  inclusive=True)
riverside_guess = missing_county[long_boolean & lat_boolean]

In [302]:
print(len(missing_county),len(riverside_guess))

417 376


Since the missing county coordinates mostly fall within Riverside county, let's plug in for the NaNs

In [309]:
t.county = t.county.replace('Riverside', 'Riverside County')
t['county'][list(riverside_guess.index)] = 'Riverside County'
t.county.dropna(inplace = True)

<ipython-input-309-c56ff6d9ec14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['county'][list(riverside_guess.index)] = 'Riverside County'


We'll have to perform a merge that matches up both the date and county for the dataframe t. 


In [316]:
t.created_at = pd.to_datetime(t.created_at) #convert the data column to timestamp for merge to compare
t = t.rename(columns = {"created_at": "date"}) #rename for cols to match

AttributeError: 'DataFrame' object has no attribute 'created_at'

# Merge Tweets and Mobility

In [311]:
tweet_mobility = t.merge(ca, how = 'left', on = ['county', 'date'])

In [318]:
len(tweet_mobility)

17965

In [320]:
tweet_mobility.head()

,date,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,...,latitude,state,county,retail_rec,groc_pharm,parks,transit,workplaces,residential,public
0,2020-03-20,Been having so much fun with these live stream...,"[33.98902523, -118.35189342]","{'id': '3b77caf94bfc81fe', 'url': 'https://api...",False,"[0, 237]","{'id': 202073462, 'id_str': '202073462', 'name...",1241080025371144192,"{'hashtags': [{'text': 'live', 'indices': [168...",False,...,33.989025,California,Los Angeles County,-40.0,-5.0,-29.0,-44.0,-41.0,22.0,-32.50
1,2020-03-20,Should I get myself some Corona? @ CVS Pharmac...,"[37.67867687, -120.95931306]","{'id': 'e41805d7248dbf1e', 'url': 'https://api...",False,"[0, 71]","{'id': 19253082, 'id_str': '19253082', 'name':...",1241075581598433281,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,...,37.678677,California,Stanislaus County,-33.0,1.0,3.0,-19.0,-32.0,14.0,-20.75
2,2020-03-20,Forced vacation? Social distancing? Unemployed...,"[33.8362, -117.89]","{'id': '0c2e6999105f8070', 'url': 'https://api...",False,"[0, 236]","{'id': 767209483, 'id_str': '767209483', 'name...",1241068075425857541,"{'hashtags': [{'text': 'chillin', 'indices': [...",True,...,33.836200,California,Orange County,-42.0,-3.0,-22.0,-46.0,-44.0,24.0,-33.75
3,2020-03-20,"If you have to leave your house for supplies, ...","[38.5763, -121.4927]","{'id': 'b71fac2ee9792cbe', 'url': 'https://api...",False,"[0, 231]","{'id': 1359353262, 'id_str': '1359353262', 'na...",1241065882694156288,"{'hashtags': [{'text': 'bronzed', 'indices': [...",False,...,38.576300,California,Sacramento County,-44.0,-7.0,-14.0,-49.0,-45.0,20.0,-36.25
4,2020-03-20,We just got baby chicks🐥🐤\nWe'll be having mor...,"[38.44279, -121.94901]","{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api...",False,"[0, 220]","{'id': 4113352096, 'id_str': '4113352096', 'na...",1241058413754417153,"{'hashtags': [{'text': 'lockewoodacres', 'indi...",False,...,38.442790,California,Solano County,-36.0,-3.0,-6.0,-32.0,-39.0,19.0,-27.50


In [322]:
tw = tweet_mobility.drop(columns = ['place', 'lang'])

In [262]:
tw.set_index('id')

,date,full_text,geo,truncated,display_text_range,user,entities,possibly_sensitive,longitude,latitude,state,county,retail_rec,groc_pharm,parks,transit,workplaces,residential,public
id,,,,,,,,,,,,,,,,,,,
1240790800088403968,2020-03-20,"My office, staying at home, helping Stop the C...","[33.6638, -118.0076]",False,"[0, 132]","{'id': 29916709, 'id_str': '29916709', 'name':...","{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.007600,33.663800,California,Orange County,-42.0,-3.0,-22.0,-46.0,-44.0,24.0,-33.75
1240791066011488257,2020-03-20,Corona CA Thu Mar 19th PM Forecast: TONIGHT Ch...,"[33.8752, -117.5655]",False,"[0, 94]","{'id': 354691403, 'id_str': '354691403', 'name...","{'hashtags': [], 'symbols': [], 'user_mentions...",True,-117.565500,33.875200,California,Riverside County,-33.0,4.0,-35.0,-30.0,-38.0,18.0,-24.25
1240796072735375366,2020-03-20,“COLDER” should i shoot this video next??? @ C...,"[33.8753, -117.566]",False,"[0, 87]","{'id': 867752113794818048, 'id_str': '86775211...","{'hashtags': [], 'symbols': [], 'user_mentions...",False,-117.566000,33.875300,California,Riverside County,-33.0,4.0,-35.0,-30.0,-38.0,18.0,-24.25
1240799104009150464,2020-03-20,.\n👆🏾ClickFollow &amp;/or a “😂”👆🏾 \n#DM to 5 p...,"[34.0522, -118.243]",False,"[0, 235]","{'id': 23940847, 'id_str': '23940847', 'name':...","{'hashtags': [{'text': 'DM', 'indices': [34, 3...",False,-118.243000,34.052200,California,Los Angeles County,-40.0,-5.0,-29.0,-44.0,-41.0,22.0,-32.50
1240804304463822849,2020-03-20,I’ll be on Hotepish tonight with my boy @dwann...,"[34.0522, -118.243]",False,"[0, 239]","{'id': 573197933, 'id_str': '573197933', 'name...","{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.243000,34.052200,California,Los Angeles County,-40.0,-5.0,-29.0,-44.0,-41.0,22.0,-32.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252381368119525383,2020-04-20,A little ray of sunshine on these dark days #c...,"[33.88366, -117.5646]",False,"[0, 202]","{'id': 74402020, 'id_str': '74402020', 'name':...","{'hashtags': [{'text': 'cheflife', 'indices': ...",False,-117.564600,33.883660,California,Riverside County,-52.0,-15.0,-48.0,-40.0,-44.0,18.0,-37.75
1252382207793983495,2020-04-20,#snapchat👻 addiction #covid #quarantinelife #s...,"[34.17541248, -118.33602388]",False,"[0, 96]","{'id': 156361436, 'id_str': '156361436', 'name...","{'hashtags': [{'text': 'snapchat', 'indices': ...",False,-118.336024,34.175412,California,Los Angeles County,-54.0,-22.0,-38.0,-50.0,-49.0,22.0,-43.75
1252382248411660288,2020-04-20,"🦋 On a #beautiful, #sunny #spring day, #strang...","[38.5763, -121.4927]",False,"[0, 201]","{'id': 114578117, 'id_str': '114578117', 'name...","{'hashtags': [{'text': 'beautiful', 'indices':...",False,-121.492700,38.576300,California,Sacramento County,-49.0,-15.0,-6.0,-55.0,-49.0,19.0,-42.00


In [256]:
tw.to_csv('../data/tweet_mobility.csv')